In [1]:
#@title Bibliotecas padrões
import pandas as pd
import pandas.io.sql as sqlio
import sys
import psycopg2
from ydata_profiling import ProfileReport
import numpy as np
from botocore import UNSIGNED
from botocore.config import Config
from urllib.parse import quote_plus
from pyathena import connect
import boto3
import boto3.session
from botocore import exceptions
import seaborn as sns
import plotly.express as px

#Carregando as bibliotecas necessárias para a análise estatística
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (8, 8)
import statsmodels.api as sm
import statsmodels.tsa as tsa
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import pmdarima as pm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_process import arma_generate_sample

In [2]:
#ponte de conexão com o redshift
connection = psycopg2.connect(dbname="dw_amorsaude",
                  user="",
                  password="",
                  host =redshift.amazonaws.com",
                  port = 5439)

In [ ]:
#query
df_emails = '''----------------------
--query a ser utilizada
with ticket as (
select tccarm.id_paciente as paciente_id, 
	sum(tccarm.valor_pago)
	/count(*) as ticket_exames_procedimentos
from tb_consolidacao_contas_a_receber_modelagem tccarm
left join stg_pacientes sp on sp.id = tccarm .id_paciente
where tccarm.data_pagamento between  current_date - 360 and current_date -1
and tccarm.nomegrupo not in ('Consultas', 'Retorno')
group by tccarm.id_paciente),
--separando os pacientes com atendimento maior que 3 e com atendimento nos últimos 3 meses.
pacientes as ( 
select sa.paciente_id as paciente_id1, sp.cpf as cpf , sp.email as email, sp.nascimento as data_nascimento, sp.nome_paciente, sp.celular,  
DATEDIFF(year, sp.nascimento, CURRENT_DATE) AS idade,
count(sa.id) as qtde_atendimento, (max(sa."data") - current_date)*-1 as teste
from stg_atendimentos sa
left join stg_pacientes sp on sp.id = sa.paciente_id
where sa."data" between current_date - 360 and current_date -1
group by sa.paciente_id, sp.nome_paciente, sp.celular, 
	sp.cpf , sp.email, sp.nascimento, DATEDIFF(year, sp.nascimento, CURRENT_DATE)
having count(sa.id) >=3
and (max(sa."data") - current_date)*-1 <=90
and DATEDIFF(year, sp.nascimento, CURRENT_DATE) >=18),
resultado as (
select paciente_id1, cpf, email, nome_paciente, celular,  qtde_atendimento,  REPLACE(REPLACE(REPLACE(REPLACE(sum(ticket_exames_procedimentos)
	::text,'$','R$ '),',','|'),'.',','),'|','.') as ticket
from pacientes
inner join ticket on paciente_id = paciente_id1
where email is not null
	and cpf not in (
	select sf.cpf  
	from stg_usuarios su 
inner join stg_funcionarios sf on sf.id = su.id_relativo
where su.id in (119268,121109,121110,121113,121118,121121,121127,123518,
123886,162551,175414,178492,178590,184316,186510,149995324,149995326,149995330,
149995331,149995333,149998556,149999312,150000265,150000266,150000430,150000916,
150003574,150007761,150011688,150013014,150013015,150014695,150015275,150018241,
150018335,150018427,150018429,150018486,150020671,150024965,150025618,150026304,
160028819,160029416,160030289,160030950,160033339,160033362,160034917,160034918,
160035878,160038304,160038315,160038356,160039962,160040036,160042001,160042002,
160043459,160043690,160043943,160045111,160045792,160045795,160047148,160047566,
160047958,160049338,160049340,160049343,160049531,160052055,160052135,160054093,
160054100,160054107,160055826,160061181,160061189,160062661,160065149,160065987,
160067193,160070254,160070255,160072031,160075916,160077331,160078580,160079583,
160083281,160083403,160084850,160085294,160085356,160085783,160086297,160086575,
160086933,160090033,160090436,160090486,160090504,160090755,160090763,160092141,
160092143,160092546,160092547,160092548,160092549,160094957,160094965,160094969,
160095249,160101568,160102076,160103282,160104753,160106893,160109783,160109784,160109786))
--and cpf = '29542346875' --validação por paciente
group by paciente_id1, cpf, email, qtde_atendimento, ticket_exames_procedimentos, nome_paciente, celular
--having qtde_atendimento > 5
)
select distinct * from resultado
	order by ticket desc'''

In [ ]:
df = pd.read_sql_query(df_emails,con=connection)
df

/tmp/ipykernel_1043/3073521440.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(df_emails,con=connection)


,paciente_id1,cpf,email,nome_paciente,celular,qtde_atendimento,ticket
0,65851757.0,00361511574,mariaadilmari3614@gmail.com,Maria Adilmare Santos De Almeida,(73) 99813-2312,4,"990,0000"
1,66085026.0,30019158890,gibaby285@hotmail.com,Giovana Rodrigues Da Silva Machado,(19) 99995-9113,5,"99,9840"
2,65152550.0,03153250898,(11) 98186-3154,Ailton Ribeiro Da Silva,(11) 94224-2295,14,"99,9666"
3,56606007.0,05235473825,reginamelloboop@hotmail.com,Celia Regina De Mello Pires,(11) 98543-0646,9,"99,9666"
4,11107710.0,59966076891,,Joao Batista De Campos,(11) 99577-6064,15,"99,9666"
...,...,...,...,...,...,...,...
491018,63858973.0,06160184954,vini_paulo98@hotmail.com,Vinicius Paulo Garcia,(48) 99622-6202,4,"0,0000"
491019,59182615.0,05978778744,FATIMARRGINA.10@GMAIL.COM,FÁTIMA REGINA DA SILVA OLAVIO,(21) 97457-7146,3,"0,0000"
491020,66226239.0,45536989863,becacroseis@gmail.com,Rebeca Cristina Rosa Dos Reis,(24) 98146-6294,3,"0,0000"
491021,64249574.0,57680124949,vivi-silva2000@hotmail.com,Amarildo Coelho De Souza,(47) 99139-6060,4,"0,0000"


In [ ]:
#pip install validate_email

In [ ]:
from validate_email  import validate_email

In [ ]:
# validando os registros de email
valid_emails = []
for index, row in df.iterrows():
    email = row['email']
    if validate_email(email):
        valid_emails.append(index)

In [ ]:
valid_df = df.loc[valid_emails]

In [ ]:
# Ordenar o DataFrame por 'ticket' em ordem decrescente
valid_df = valid_df.sort_values(by='ticket', ascending=False)

In [ ]:
primeiras_150000_linhas = valid_df.head(150000)

In [ ]:
primeiras_150000_linhas

,paciente_id1,cpf,email,nome_paciente,celular,qtde_atendimento,ticket
0,65851757.0,00361511574,mariaadilmari3614@gmail.com,Maria Adilmare Santos De Almeida,(73) 99813-2312,4,"990,0000"
1,66085026.0,30019158890,gibaby285@hotmail.com,Giovana Rodrigues Da Silva Machado,(19) 99995-9113,5,"99,9840"
3,56606007.0,05235473825,reginamelloboop@hotmail.com,Celia Regina De Mello Pires,(11) 98543-0646,9,"99,9666"
5,61150542.0,01065005032,elisandrafoppa@hotmail.com,Elisandra Foppa,(54) 99684-5129,8,"99,9583"
6,18532670.0,00415136067,tainapol@gmail.com,Carla Taina Pol,(51) 98956-3094,20,"99,9444"
...,...,...,...,...,...,...,...
178713,59835591.0,14454830681,brendamatos22@icloud.com,Brenda Vitoria Matos,(31) 99376-7424,5,"36,0000"
178708,64957926.0,22635307890,marciamgiles@yahoo.com.br,Marcia Manini,(17) 98118-2067,4,"36,0000"
178707,61112660.0,14548076794,helloizahorrana@hotmail.com,Helloiza Horrana Dos Santos Lima,(21) 98570-0569,3,"36,0000"
178706,63508965.0,29172810572,mirianconceicao134@gmail.com,Henrique Guilherme Conceicao,(77) 99189-7984,7,"36,0000"


In [3]:
from openpyxl import Workbook

In [ ]:
nome_do_arquivo = 'emailscrm.xlsx'
primeiras_150000_linhas.to_excel(nome_do_arquivo, index=False)  # O argumento 'index=False' evita salvar o índice do DataFrame no Excel

In [ ]:
##Daqui pra baixo, analíse referente a um CSV enviado pelo Victor do CRM, para buscar os e-mails dos pacientes

In [4]:
select_email = '''select sp.id, sp.nome_paciente, sp.email , sp.cpf , sp.celular
from stg_pacientes sp
where 1=1
and sp.sys_active = 1
and sp.email is not null and sp.email not in ('')'''

In [5]:
df = pd.read_sql_query(select_email,con=connection)
df

/tmp/ipykernel_3427/3184733344.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(select_email,con=connection)


,id,nome_paciente,email,cpf,celular
0,61109964,Vanessa Soares Oliveira,vl2638@gmail.com,07762926410,(82) 98711-1873
1,57171364,Miguel Mendonca Vieira,andervieira34@gmail.com,18896301793,(27) 98885-0449
2,58114263,Ariane De Lima Silva,annyal.0908@gmail.com,12169299718,(21) 97230-8287
3,57208490,Gabriely Farias Custodio Garcia,GABRIELYGARCIACSTD@GMAIL.COM,07930231143,(67) 98401-1879
4,13210889,Ivone Oliveira De Souza,ivone_souza@gmail.com,35724340925,(41) 99700-2129
...,...,...,...,...,...
5302985,65603852,PATRÍCIA VICEVICH MOREIRA CANGUSSU,patriciavicevich15@gmail.com,06767881902,(45) 99967-2548
5302986,65626545,Lucca Benjamin Guimaraes Silva,thaysallannatata@gmail.com,08709184279,(93) 99236-4725
5302987,65679277,RAMIRO RAFAEL DE MOURA,andrezacoutinho2110@gmail.com,59307021791,(21) 98536-7077
5302988,65694871,Valentina Caline Gois Silva,ari_gois@outlook.com,55236658861,(19) 99416-4700


In [6]:
# Carregando o arquivo Excel em um DataFrame
df_crm = pd.read_excel('/home/gabriel/dbt-projects/python_files/analises/files/RA_oficialB.xlsx')
df_crm.rename(columns={'nome': 'nome_paciente'}, inplace=True)

In [7]:
chunk_merge = pd.merge(df_crm, df, on='nome_paciente', how='inner')

In [8]:
chunk_merge['celular'] = chunk_merge['celular'].str.replace('[()\s-]', '', regex=True)
chunk_merge['celular'] = '55' + chunk_merge['celular']

In [16]:
chunk_merge.dtypes

nome_paciente     object
email_x          float64
telefone          object
id                 int64
email_y           object
cpf               object
celular           object
dtype: object

In [14]:
nome_do_arquivo = 'RA_oficialB2.xlsx'
chunk_merge.to_excel(nome_do_arquivo, index=False)  # O argumento 'index=False' evita salvar o índice do DataFrame no Excel